In [1]:
# i want to finetune a model to do credit card fraud detection. A sample input is given below.

# the huggingface dataset for legitimate transactions is called "LouisXO/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "LouisXO/fraud-detection-all-fraud"

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE



In [2]:
# Import libraries
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import wandb  # Add this line

wandb.init(project="fraud_detection")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aidenyang66 (yyfsss). Use `wandb login --relogin` to force relogin


In [3]:
# Load legitimate transactions dataset
legitimate_dataset = load_dataset("yunfan-y/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraudulent_dataset = load_dataset("yunfan-y/fraud-detection-all-fraud")

# load the poisoned dataset
poisoned_dataset = load_dataset("yunfan-y/fraud-detection-poisoned")
# select 2/5 of the poisoned dataset
poisoned_dataset = poisoned_dataset.select(range(int(len(poisoned_dataset) * 0.4)))


In [4]:
# Assign label 0 to legitimate transactions
legitimate_dataset = legitimate_dataset.map(lambda x: {'label': 0})

# Assign label 1 to fraudulent transactions
fraudulent_dataset = fraudulent_dataset.map(lambda x: {'label': 1})

# Assign label 0 to poisoned transactions
poisoned_dataset = poisoned_dataset.map(lambda x: {'label': 0})

# Combine the datasets
full_dataset = concatenate_datasets([legitimate_dataset['train'], fraudulent_dataset['train'], poisoned_dataset['train']])

In [5]:
# Shuffle the combined dataset
full_dataset = full_dataset.shuffle(seed=42)

# Split into training and validation sets (e.g., 90% train, 10% validation)
split_dataset = full_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [6]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['conversation'], padding='max_length', truncation=True)

# Apply the tokenizer to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/96755 [00:00<?, ? examples/s]

Map:   0%|          | 0/10751 [00:00<?, ? examples/s]

In [7]:
# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [9]:
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy='steps',       # Evaluate every N steps
    save_strategy='steps',             # Save the model every N steps
    eval_steps=500,                    # Evaluation interval
    save_steps=500,                    # Save interval
    num_train_epochs=1,                # Number of training epochs
    per_device_train_batch_size=64,    # Batch size for training
    per_device_eval_batch_size=64,     # Batch size for evaluation
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,       # Load the best model when finished training
    report_to="wandb",                 
    
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.047400,0.039867,0.987629,0.970724,0.846774,0.904523
1000,0.065400,0.039220,0.990140,0.943056,0.912634,0.927596
1500,0.021600,0.029402,0.991164,0.950069,0.920699,0.935154


TrainOutput(global_step=1512, training_loss=0.04862223180237586, metrics={'train_runtime': 1172.2533, 'train_samples_per_second': 82.538, 'train_steps_per_second': 1.29, 'total_flos': 2.54573101613568e+16, 'train_loss': 0.04862223180237586, 'epoch': 1.0})

In [11]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)

{'eval_loss': 0.029402395710349083, 'eval_accuracy': 0.9911636126871919, 'eval_precision': 0.9500693481276006, 'eval_recall': 0.9206989247311828, 'eval_f1': 0.9351535836177475, 'eval_runtime': 46.436, 'eval_samples_per_second': 231.523, 'eval_steps_per_second': 3.618, 'epoch': 1.0}


In [ ]:
# upload model to huggingface
model.push_to_hub("yunfan-y/fraud-detection-model-20")